In [1]:
import warnings
warnings.filterwarnings("ignore")

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Concatenate, LSTM, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import plot_model

import pandas as pd
import numpy as np

import re
np.random.seed(5)

from sklearn import preprocessing
from sklearn.metrics import log_loss, classification_report

In [2]:
# Análise de reviews do Smartwatch Amazfit Bip U Pro - Preto -  Amazon
df_texto = pd.read_csv('run_results2.csv')
df_texto.head(2)

,controle_review_curto,controle_review_longo,controle_nota,controle_nota_url,controle_selection1
0,Mais que recomendo!!!!,Que sensacional! O modelo da minha tv é uma da...,"5,0 de 5 estrelas",https://www.amazon.com.br/gp/customer-reviews/...,Avaliado no Brasil em 10 de maio de 2021
1,Não cumpre plenamente o que se propõe,Devo dizer que estou bem decepcionado.\nCompre...,"2,0 de 5 estrelas",https://www.amazon.com.br/gp/customer-reviews/...,Avaliado no Brasil em 13 de julho de 2021


In [3]:
df_texto.tail(2)

,controle_review_curto,controle_review_longo,controle_nota,controle_nota_url,controle_selection1
1508,Funcionalidades confirmadas.,O aparelho é realmente aquilo que esta descrit...,"5,0 de 5 estrelas",https://www.amazon.com.br/gp/customer-reviews/...,Avaliado no Brasil em 14 de fevereiro de 2022
1509,Não vale a pena.,Aparente parece um ótimo produto. Porém tem mu...,"1,0 de 5 estrelas",https://www.amazon.com.br/gp/customer-reviews/...,Avaliado no Brasil em 27 de janeiro de 2022


In [4]:
df_texto.drop('controle_nota_url', axis=1, inplace= True)
df_texto.head(2)

,controle_review_curto,controle_review_longo,controle_nota,controle_selection1
0,Mais que recomendo!!!!,Que sensacional! O modelo da minha tv é uma da...,"5,0 de 5 estrelas",Avaliado no Brasil em 10 de maio de 2021
1,Não cumpre plenamente o que se propõe,Devo dizer que estou bem decepcionado.\nCompre...,"2,0 de 5 estrelas",Avaliado no Brasil em 13 de julho de 2021


In [5]:
# Pré-processando os dados de datas.
meses=['janeiro',' fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']
mesess = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
numeros=['0','1','2','3','4','5','6','7','8','9']
i=0
df_texto['data']=1

for datas in df_texto['controle_selection1']:
    if datas[23] in numeros:
        dia = datas[22:24]
    else:
        dia = datas[22]
    for mese in meses:
        if datas[28-len(datas):-8] in mese:
            mes = mesess[meses.index(mese)]
    ano = datas[-4:len(datas)]
    if '202' in datas:
        if '2021' in datas:
            ano = '2021'
        elif '2022' in datas:
            ano = '2022'
    
    df_texto['data'].iloc[i,:]=dia + "-" + mes + "-" + ano
    i=i+1

In [6]:
df_texto.head(60)

,controle_review_curto,controle_review_longo,controle_nota,controle_selection1,data
0,Mais que recomendo!!!!,Que sensacional! O modelo da minha tv é uma da...,"5,0 de 5 estrelas",Avaliado no Brasil em 10 de maio de 2021,10-05-2021
1,Não cumpre plenamente o que se propõe,Devo dizer que estou bem decepcionado.\nCompre...,"2,0 de 5 estrelas",Avaliado no Brasil em 13 de julho de 2021,13-07-2021
2,Não tem integração com toda TV,Eu comprei esse modelo justamente pelos botões...,"3,0 de 5 estrelas",Avaliado no Brasil em 15 de maio de 2021,15-05-2021
3,"O PRODUTO É ÓTIMO, MAS A ENTREGA FOI MEDONHA!!!","Pow, Amazon, que isso? vcs eram tão competente...","1,0 de 5 estrelas",Avaliado no Brasil em 27 de setembro de 2021,27-09-2021
4,Versátil,Pontos positivos: o processo de instalação é b...,"5,0 de 5 estrelas",Avaliado no Brasil em 17 de maio de 2021,17-05-2021
5,Muito Bom e rápido!,"Teclado na tela não é QWERT, e também o navega...","4,0 de 5 estrelas",Avaliado no Brasil em 7 de maio de 2021,7-05-2021
6,"ótimo produto, péssima logística para entrega ...","Gostei muito do produto, achei somente que o c...","1,0 de 5 estrelas",Avaliado no Brasil em 18 de agosto de 2021,18-08-2021
7,A Alexa não nos escuta se não apertamos o botã...,Sempre via em vídeos o pessoal falando com a A...,"5,0 de 5 estrelas",Avaliado no Brasil em 23 de outubro de 2021,23-10-2021
8,Fire Stick trava e conexão com Echo Studio é p...,O Fire Stick trava (aconteceu várias vezes) e ...,"3,0 de 5 estrelas",Avaliado no Brasil em 4 de julho de 2021,4-07-2021
9,ÓTIMA ESCOLHA!,Muito contente com o FIRE TV STICK da Amazon. ...,"5,0 de 5 estrelas",Avaliado no Brasil em 16 de novembro de 2021,16-11-2021


In [7]:
i=0
df_texto['nota']=1
for notas in df_texto['controle_nota']:
    df_texto['nota'].iloc[i,:]=notas[:1]
    i=i+1
    
df_texto.head(10)

,controle_review_curto,controle_review_longo,controle_nota,controle_selection1,data,nota
0,Mais que recomendo!!!!,Que sensacional! O modelo da minha tv é uma da...,"5,0 de 5 estrelas",Avaliado no Brasil em 10 de maio de 2021,10-05-2021,5
1,Não cumpre plenamente o que se propõe,Devo dizer que estou bem decepcionado.\nCompre...,"2,0 de 5 estrelas",Avaliado no Brasil em 13 de julho de 2021,13-07-2021,2
2,Não tem integração com toda TV,Eu comprei esse modelo justamente pelos botões...,"3,0 de 5 estrelas",Avaliado no Brasil em 15 de maio de 2021,15-05-2021,3
3,"O PRODUTO É ÓTIMO, MAS A ENTREGA FOI MEDONHA!!!","Pow, Amazon, que isso? vcs eram tão competente...","1,0 de 5 estrelas",Avaliado no Brasil em 27 de setembro de 2021,27-09-2021,1
4,Versátil,Pontos positivos: o processo de instalação é b...,"5,0 de 5 estrelas",Avaliado no Brasil em 17 de maio de 2021,17-05-2021,5
5,Muito Bom e rápido!,"Teclado na tela não é QWERT, e também o navega...","4,0 de 5 estrelas",Avaliado no Brasil em 7 de maio de 2021,7-05-2021,4
6,"ótimo produto, péssima logística para entrega ...","Gostei muito do produto, achei somente que o c...","1,0 de 5 estrelas",Avaliado no Brasil em 18 de agosto de 2021,18-08-2021,1
7,A Alexa não nos escuta se não apertamos o botã...,Sempre via em vídeos o pessoal falando com a A...,"5,0 de 5 estrelas",Avaliado no Brasil em 23 de outubro de 2021,23-10-2021,5
8,Fire Stick trava e conexão com Echo Studio é p...,O Fire Stick trava (aconteceu várias vezes) e ...,"3,0 de 5 estrelas",Avaliado no Brasil em 4 de julho de 2021,4-07-2021,3
9,ÓTIMA ESCOLHA!,Muito contente com o FIRE TV STICK da Amazon. ...,"5,0 de 5 estrelas",Avaliado no Brasil em 16 de novembro de 2021,16-11-2021,5


In [8]:
df_texto.dtypes

controle_review_curto    object
controle_review_longo    object
controle_nota            object
controle_selection1      object
data                     object
nota                      int64
dtype: object

In [9]:
df = df_texto[['controle_review_longo','data','nota']]
df.head(10)

,controle_review_longo,data,nota
0,Que sensacional! O modelo da minha tv é uma da...,10-05-2021,5
1,Devo dizer que estou bem decepcionado.\nCompre...,13-07-2021,2
2,Eu comprei esse modelo justamente pelos botões...,15-05-2021,3
3,"Pow, Amazon, que isso? vcs eram tão competente...",27-09-2021,1
4,Pontos positivos: o processo de instalação é b...,17-05-2021,5
5,"Teclado na tela não é QWERT, e também o navega...",7-05-2021,4
6,"Gostei muito do produto, achei somente que o c...",18-08-2021,1
7,Sempre via em vídeos o pessoal falando com a A...,23-10-2021,5
8,O Fire Stick trava (aconteceu várias vezes) e ...,4-07-2021,3
9,Muito contente com o FIRE TV STICK da Amazon. ...,16-11-2021,5


In [10]:
# Transformando tipo de dados das colunas
df['data'] = pd.to_datetime(df['data'], format="%d-%m-%Y")
df['nota'] =  df['nota'].astype(int)
df.dtypes

controle_review_longo            object
data                     datetime64[ns]
nota                              int32
dtype: object

In [11]:
df.tail(30)

,controle_review_longo,data,nota
1480,Só não dou 5 estrela pq infelizmente não tem o...,2021-12-27,4
1481,Gostaria de ter um controle que possa ter aces...,2021-10-17,2
1482,O Fire Stick TV é o melhor para quem quer ter ...,2022-01-25,5
1483,"Imagem excelente, fácil de instalar, áudio bom...",2021-05-08,5
1484,"Simplesmente sensacional, chegou antes da Data...",2022-01-21,5
1485,Gostei muito do produto. Rápido e intuitivo.\n...,2021-05-11,4
1486,ótimo aparelho. Apesar de não ter o app da Glo...,2022-01-02,3
1487,Pra quem não quer comprar uma tv nova essa é u...,2022-01-23,5
1488,"De vez em quando dá uma travadinha, o que pode...",2021-10-11,4
1489,"Eu amei o fire stick, só tive problemas em ach...",2022-01-30,5


In [12]:
# Separando dados por niveis
escala = [
    (df["nota"] < 3),
    (df["nota"] == 3),
    (df["nota"] >= 4)
         ]
nivel = ["Negativo", "Neutro", "Positivo"]

df["Notas"] = np.select(escala, nivel)
df.head()

,controle_review_longo,data,nota,Notas
0,Que sensacional! O modelo da minha tv é uma da...,2021-05-10,5,Positivo
1,Devo dizer que estou bem decepcionado.\nCompre...,2021-07-13,2,Negativo
2,Eu comprei esse modelo justamente pelos botões...,2021-05-15,3,Neutro
3,"Pow, Amazon, que isso? vcs eram tão competente...",2021-09-27,1,Negativo
4,Pontos positivos: o processo de instalação é b...,2021-05-17,5,Positivo


#  Aplicando CNN

Preprarando os dados de reviews

In [13]:
# Convertendo as avaliações
def converteCategoria(df, coluna):
    le = preprocessing.LabelEncoder()
    le.fit(df[coluna])
    df[coluna] = le.transform(df[coluna])
    return le

labelEncoderNotas = converteCategoria(df, 'Notas')
df.head()

,controle_review_longo,data,nota,Notas
0,Que sensacional! O modelo da minha tv é uma da...,2021-05-10,5,2
1,Devo dizer que estou bem decepcionado.\nCompre...,2021-07-13,2,0
2,Eu comprei esse modelo justamente pelos botões...,2021-05-15,3,1
3,"Pow, Amazon, que isso? vcs eram tão competente...",2021-09-27,1,0
4,Pontos positivos: o processo de instalação é b...,2021-05-17,5,2


Codificando o texto em linguagem natural

In [14]:
# Toquenização
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

tokenized_sents = [word_tokenize(i) for i in df['controle_review_longo'].values]

# Representação de palavras usando o Word2Vec
from gensim.models import Word2Vec
model_review = Word2Vec(tokenized_sents, vector_size=100, window=5, min_count=1, workers=4)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wfs4.NB555-WFS4\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
# Separando dados de entrado e de saída (labels)
X, y = [], []
max_len = 0

for i, row in df.iterrows():
    vetores = model_review.wv[word_tokenize(row['controle_review_longo'])]

    max_len = max(max_len, len(vetores))
    X.append(vetores)
    y.append(row['Notas'])
print("max_len: {}".format(max_len))

max_len: 326


In [16]:
def transform(exemplos, dimension):
    results = np.zeros((len(exemplos), dimension, 100))
    for i, sequence in enumerate(exemplos):
        results[i, :len(sequence), :] = sequence
    return results
 
X = transform(X, max_len)
y = np.array(y).astype("float32")

In [17]:
np.unique(y)

array([0., 1., 2.], dtype=float32)

In [18]:
X.shape

(1510, 326, 100)

In [19]:
max_len

326

Dividindo dados em treino e teste

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

Difinindo a rede

In [21]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

model = Sequential()
model.add(Flatten(input_shape=(max_len, 100)))
model.add(Dense(1000, activation='relu')),
model.add(Dropout(0.3))
model.add(Dense(1000, activation='relu')),
model.add(Dropout(0.3))
model.add(Dense(1000, activation='relu')),
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 32600)             0         
                                                                 
 dense (Dense)               (None, 1000)              32601000  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 1000)              1001000   
                                                                 
 dropout_1 (Dropout)         (None, 1000)              0         
                                                                 
 dense_2 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_3 (Dense)             (None, 3)                 3

In [22]:
model.compile(optimizer = "adam",
              loss = "sparse_categorical_crossentropy",
              metrics = ["accuracy"]
              )

In [23]:
results = model.fit(X_train, y_train,
                    epochs= 5,
                    batch_size = 100,
                    validation_data = (X_test, y_test)
                    )

Epoch 1/5
11/11 [==============================] - 2s 161ms/step - loss: 1.8045 - accuracy: 0.7237 - val_loss: 1.0731 - val_accuracy: 0.6490
Epoch 2/5
11/11 [==============================] - 2s 142ms/step - loss: 0.7299 - accuracy: 0.8013 - val_loss: 0.6471 - val_accuracy: 0.8675
Epoch 3/5
11/11 [==============================] - 1s 137ms/step - loss: 0.5893 - accuracy: 0.8316 - val_loss: 0.5584 - val_accuracy: 0.8631
Epoch 4/5
11/11 [==============================] - 1s 136ms/step - loss: 0.5894 - accuracy: 0.8316 - val_loss: 0.5469 - val_accuracy: 0.8631
Epoch 5/5
11/11 [==============================] - 2s 140ms/step - loss: 0.5858 - accuracy: 0.8288 - val_loss: 0.5515 - val_accuracy: 0.8675


LSTM

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

n_most_common_words = 8000
max_len = 326
tokenizer = Tokenizer(num_words=n_most_common_words, 
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', 
                      lower=True)
tokenizer.fit_on_texts(df['controle_review_longo'].values)
sequences = tokenizer.texts_to_sequences(df['controle_review_longo'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len, padding='post')

Found 6331 unique tokens.


In [26]:
y = np.array(y).astype("int64")
labels = to_categorical(y, num_classes=3)
print(labels[:10])

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25, random_state=42)

In [28]:
epochs = 10
emb_dim = 128
batch_size = 256
labels[:2]

array([[0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [40]:
from keras.callbacks import EarlyStopping

print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))

model = Sequential()
model.add(Embedding(n_most_common_words, emb_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["acc"])
print(model.summary())
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                    validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',
                    patience=7, min_delta=0.001)])

((1132, 326), (1132, 3), (378, 326), (378, 3))
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 326, 128)          1024000   
                                                                 
 spatial_dropout1d_6 (Spatia  (None, 326, 128)         0         
 lDropout1D)                                                     
                                                                 
 lstm_6 (LSTM)               (None, 64)                49408     
                                                                 
 dense_10 (Dense)            (None, 3)                 195       
                                                                 
Total params: 1,073,603
Trainable params: 1,073,603
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
4/4 [==============================] - 11s 2s/s

In [31]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

12/12 [==============================] - 1s 81ms/step - loss: 0.4743 - accuracy: 0.8704
Test set
  Loss: 0.474
  Accuracy: 0.870
